In [1]:
from google.oauth2 import service_account
from google.cloud import storage
from pathlib import Path
import datetime
import os

In [50]:
video_title = 'Eifel Grand Prix - Recap'

video_url = "https://www.youtube.com/watch?v=I84eb-B0l94"

path_data = Path('../data/')
path_json = path_data / 'logo-detection.json'
pathIn = path_data / str(video_title + '/input/')
pathIn_Video = pathIn / str(video_title + '.mp4')
pathIn_Frames =  pathIn / 'frames'
pathIn_Frames_Resized = pathIn / 'frames_resized'

pathOut = path_data / str(video_title + '/output/')
path_annotations = pathOut / 'annotations.csv'
path_logos_video = pathOut / 'Logos.mp4'

In [9]:
credentials = service_account.Credentials.from_service_account_file(path_json)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(path_json)

In [10]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client(credentials=credentials)

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [11]:
def create_bucket_class_location(bucket_name):
    """Create a new bucket in specific location with storage class"""
    # bucket_name = "your-new-bucket-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "COLDLINE"
    new_bucket = storage_client.create_bucket(bucket, location="us")

    print(
        "Created bucket {} in {} with storage class {}".format(
            new_bucket.name, new_bucket.location, new_bucket.storage_class
        )
    )
    return new_bucket


In [28]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    #storage_client = storage.Client.from_service_account_json(str(path_json))
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name, timeout=1000)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
    
    
# def upload_blob(bucket_name, source_file_name, destination_blob_name):

#     client = storage.Client()
#     bucket = client.bucket(bucket_name)


#     # Write file if necessary.
#     blob = bucket.blob(destination_blob_name)
#     if not blob.exists():
#         print(f"Writing {destination_blob_name}")
#         start = datetime.datetime.now()
#         blob.upload_from_filename(source_file_name)
#         end = datetime.datetime.now()
#         print(f"Wrote {destination_blob_name} {end-start}")


In [13]:
implicit()

[<Bucket: videos-detection>]


In [31]:
create_bucket_class_location(f'videos-detection/videos_files/F1/{video_title}')

In [30]:
path_uri = f'videos_files/F1/{video_title}/video.mp4'
upload_blob('videos-detection', str(pathIn_Video), path_uri)

File ../data/Eifel Grand Prix - Recap/input/Eifel Grand Prix - Recap.mp4 uploaded to videos_files/F1/Eifel Grand Prix - Recap/video.mp4.


In [32]:
#upload_blob('videos-detection', str(pathIn / 'frames.zip'), 'F1 Hamilton/frames/')

In [36]:
def is_stored(bucket_name, path_uri):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    stats = storage.Blob(bucket=bucket, name=path_uri).exists(storage_client)
    return stats

In [37]:
is_stored(bucket_name, path_uri)

True

In [39]:
def generate_image_url(blob_path, bucket_name):
    """ generate signed URL of a video stored on google storage. 
        Valid for 300 seconds in this case. You can increase this 
        time as per your requirement. 
    """ 
    
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_path) 
    return blob.generate_signed_url(datetime.timedelta(seconds=300), method='GET')

In [41]:
url = generate_image_url('videos_files/F1/Eifel Grand Prix - Recap/video.mp4', 'videos-detection')

In [53]:
import urllib.request as req
req.urlretrieve(video_url, str(video_title + '.mp4'))

('Eifel Grand Prix - Recap.mp4', <http.client.HTTPMessage at 0x10b49a890>)

In [46]:
bucket_name = 'videos-detection'
video_category = 'F1'
blob_path_video = f'videos_files/{video_category}/{video_title}/video.mp4'
uri_video = f'gs://{bucket_name}/'+blob_path_video

In [47]:
uri_video

'gs://videos-detection/videos_files/F1/Eifel Grand Prix - Recap/video.mp4'

In [48]:
'gs://videos-detection/videos_files/F1/Eifel Grand Prix - Recap/video.mp4'

'gs://videos-detection/videos_files/F1/Eifel Grand Prix - Recap/video.mp4'